In [1]:
import os
import numpy as np
import typing as ty

In [5]:
# Import Process level primitives
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort
from lava.magma.core.process.ports.reduce_ops.reduce_ops import ReduceSum
from lava.proc.dense.process import Dense
from lava.proc.lif.process import LearningLIF
from lava.proc.learning_rules.stdp_learning_rule import STDPLoihi

ModuleNotFoundError: No module named 'lava.magma.core.process.ports.ports.reduce_ops'; 'lava.magma.core.process.ports.ports' is not a package

In [8]:
LIFRefractory?

Init signature: LIFRefractory(*args, **kwargs)
Docstring:     
Leaky-Integrate-and-Fire (LIF) process with refractory period.

Parameters
----------
shape : tuple(int)
    Number and topology of LIF neurons.
u : float, list, numpy.ndarray, optional
    Initial value of the neurons' current.
v : float, list, numpy.ndarray, optional
    Initial value of the neurons' voltage (membrane potential).
du : float, optional
    Inverse of decay time-constant for current decay. Currently, only a
    single decay can be set for the entire population of neurons.
dv : float, optional
    Inverse of decay time-constant for voltage decay. Currently, only a
    single decay can be set for the entire population of neurons.
bias_mant : float, list, numpy.ndarray, optional
    Mantissa part of neuron bias.
bias_exp : float, list, numpy.ndarray, optional
    Exponent part of neuron bias, if needed. Mostly for fixed point
    implementations. Ignored for floating point implementations.
vth : float, optional

In [6]:
class LsnnNet(AbstractProcess):
    def __init__(self, input_weights,
                output_weights,
                recurrent_weights,
                **params) -> None:
        
        super().__init__()

        self.input = Dense(weights = input_weights)

        self.forward_lif = LearningLIF(shape=input_weights.shape[0],
                                       u = params.get("f_u",1),
                                       v = params.get("f_v",0),
                                       dv = params.get("f_dv",0),
                                       du = params.get("f_du",1),
                                       bias_exp = params.get("f_bias_exp",1),
                                       bias_mant = params.get("f_bias_mant",1),
                                       vth = params.get("f_vth",1),
                                       log_config = params.get("log_config",1),
                                       learning_rule = params.get("learning_rule", None)
                                       )
        
        self.output = Dense(weights = output_weights)

        self.backword_alif = LearningLIF(shape = output_weights.shape[0],
                                       u = params.get("r_u",1),
                                       v = 0,
                                       du = params.get("r_du",1),
                                       dv = 1,
                                       bias_exp = 0,
                                       bias_mant = 1,
                                       vth = 10000,
                                       log_config = params.get("log_config",1),
                                       learning_rule = params.get("learning_rule",None)
                                       )
          
        self.recurrent = Dense(weights = recurrent_weights)

        self.s_in  = InPort(shape = input_weights.shape[1], reduce_op = ReduceSum)
        self.a_out  = OutPort(shape = output_weights.shape[0])
        
        self.s_in.connect(self.input.s_in)
        self.output.a_out.connect(self.a_out)

        self.forward_lif.a_in.connect_from([self.input.out_ports, self.recurrent])


        self.forward_lif.s_out.connect(self.output.s_in)

        self.output.a_out.connect(self.backword_alif.a_in)
        self.backword_alif.s_out.connect(self.recurrent.s_in)

        

Init signature: Dense(*args, **kwargs)
Docstring:     
Dense connections between neurons. Realizes the following abstract
behavior: a_out = weights * s_in

Parameters
----------
weights : numpy.ndarray
    2D connection weight matrix of form (num_flat_output_neurons,
    num_flat_input_neurons) in C-order (row major).

weight_exp : int, optional
    Shared weight exponent of base 2 used to scale magnitude of
    weights, if needed. Mostly for fixed point implementations.
    Unnecessary for floating point implementations.
    Default value is 0.

num_weight_bits : int, optional
    Shared weight width/precision used by weight. Mostly for fixed
    point implementations. Unnecessary for floating point
    implementations.
    Default is for weights to use full 8 bit precision.

sign_mode : SignMode, optional
    Shared indicator whether synapse is of type SignMode.NULL,
    SignMode.MIXED, SignMode.EXCITATORY, or SignMode.INHIBITORY. If
    SignMode.MIXED, the sign of the weight is
    